# Multi-Class Perceptron for Emotion Classification 
## *(With TF-IDF Vectors)*
This notebook contains an implementation of the multi-class perceptron algorithm for emotion classification using the ISEAR dataset. Datafiles are .csv files containing texts tagged with 7 self-reported emotions: joy, fear, anger, sadness, disgust, shame, and guilt. After data cleaning, the training set contains 5186 texts and the test set contains 1117. 
## Perceptron Class
The Perceptron class is initialized with 3 parameters: X-train - List of texts for training the model; y-train - List of labels corresponding to X-train; epoch - an integer corresponding to the number of training iterations. The weight vectors are a two-dimension dictionary containing 7 weight vectors (one for each label) of vocabulary length. The initial weights are randomized to a value between 0 and 1. 

The perceptron builds feature vectors using TF-IDF. For each document in our training set, we calculate the term frequency as:

$\frac{word\ occurrances}{total\ words}$

The inverse document frequency is calculated as:

$\log{_e}{\frac{num\ docs}{docs\ containing\ word}}$

The product of these two values is used to represent each word/feature in the sentences.

With each training epoch, for each text, for each emotion label, we take the dot product of the feature vector and corresponding weight vector. The argmax of these calculations is assumed to be the predicted label. If the perceptron has accurately classified the text, we do nothing. If the classification is incorrect, we reward the correct label by adding the feature vector to its weight vector. We penalize the incorrectly predicted label by subtracting the feature vector from its weight vector.

Similarly, the predict method vectorizes a list of test data to be classified and returns a list of predictions that are the argmax of the dot product of each feature vector and each weight vector.

In [1]:
import data_utils as du
import random
import tfidf

In [3]:
# Class definition for perceptron contains methods init, train, and predict
class Perceptron:
    def __init__(self, X_train, y_train, epoch):
        self.X_train = X_train
        self.y_train = y_train
        self.epoch = epoch
        # These will be updated below
        self.weight_vecs = None
        self.labels = None

        #TF_IDF
        tf_idf_model = tfidf.Tfidf(self.X_train, Y=self.y_train, train=True)
        tf_idf_model.tf_idf()
        self.feature_vecs = tf_idf_model.tf_idf_d
        # Get unique labels
        self.labels = list(set(y_train))

        # Create weight vectors length of vocabulary for each label. This is a 2D dictionary where:
        # Keys are labels and values are dictionaries containing a weight (initialized to value between 0-1) for each feature (word) in our vocabulary.
        self.weight_vecs = {key: dict() for key in self.labels}
        for key in self.weight_vecs:
            for i in tf_idf_model.unique_tokens:
                self.weight_vecs[key][i] = random.random()
  
    def train(self):
        # We iterate over the model for the specified number of epochs
        for i in range(self.epoch):
            for sent in self.feature_vecs:
                # Initialize a dictionary to hold each dot product calculation
                emotion_scores = dict(joy=0, fear=0, guilt=0, anger=0, disgust=0, sadness=0, shame=0)
                # Calculate the dot product
                for label in self.labels:
                    dot_product = 0
                    for feature in self.feature_vecs[sent]:
                        if feature != 'LABEL':
                            dot_product += self.feature_vecs[sent][feature] * self.weight_vecs[label][feature]
                    
                    emotion_scores[label] = dot_product

                # Find the argmax
                argmax = max(emotion_scores, key=emotion_scores.get)
                correct_label = self.feature_vecs[sent]['LABEL']
                
                # Reward/Penalty update step
                if argmax != correct_label:
                    for feature in self.feature_vecs[sent]:
                        if feature != 'LABEL':
                            self.weight_vecs[correct_label][feature] += self.feature_vecs[sent][feature]
                            self.weight_vecs[argmax][feature] -= self.feature_vecs[sent][feature]
                         

    # Returns list of predicted labels given X_test parameter
    def predict(self, X_test):
        test_model = tfidf.Tfidf(X_test)
        test_model.tf_idf()
        train_vecs = test_model.tf_idf_d

        predictions = []
        for sent in train_vecs:
            emotion_scores = dict(joy=0, fear=0, guilt=0, anger=0, disgust=0, sadness=0, shame=0)
            for label in self.labels:
                dot_product = 0
                for feature in train_vecs[sent]:
                    if feature in self.weight_vecs[label]:
                        dot_product += train_vecs[sent][feature] * self.weight_vecs[label][feature]
                emotion_scores[label] = dot_product
            argmax = max(emotion_scores, key=emotion_scores.get)
            predictions.append(argmax)
        
        return predictions     

## Prep Training and Test Data
Now, we first prepare our training and test data to be used by the Perceptron class. This section defines two functions: sep_labels() and prep_data(). The first, sep_labels() decouples the labels from our training documents and returns a list of training documents, X, and a list of corresponding labels, y. The prep_data() function accepts a two arguments: a path to the .csv training data, and a path to the .csv test data. The function returns 4 lists: X_train, y_train, X_test, y_test. 

In [4]:
X_train, y_train, X_test, y_test = du.prep_data('isear-train.csv', 'isear-test.csv')

## Testing the model
We initialize the Perceptron with X_traing, y_train, and select a number of epochs. We then call model.fit() to train the model, and model.predict() to receive our list of predicted labels. 

In [13]:
model = Perceptron(X_train, y_train, epoch = 3)
model.train()

In [14]:
predictions = model.predict(X_test)

## Evaluation
We import the Evaluator class from the evaluator module. The evaluator object is initialized with a list of predictions and true values. Using the ret_fscore() method, we can see the presicion, recall, and f1score for each emotion.

In [15]:
from evaluator import Evaluator

eval = Evaluator(predictions, y_test)
eval.ret_fscore()

In [16]:
print(eval.precision)
print(eval.recall)
print(eval.f_score)
print('----------------------------------------------------')
print()
print(eval.accuracy())

{'joy': 0.6132596685082873, 'fear': 0.6219512195121951, 'guilt': 0.37894736842105264, 'anger': 0.4065040650406504, 'disgust': 0.48255813953488375, 'sadness': 0.5899280575539568, 'shame': 0.4391891891891892}
{'joy': 0.69375, 'fear': 0.6219512195121951, 'guilt': 0.4864864864864865, 'anger': 0.29239766081871343, 'disgust': 0.4911242603550296, 'sadness': 0.5503355704697986, 'shame': 0.4166666666666667}
{'joy': 0.6510263929618768, 'fear': 0.6219512195121951, 'guilt': 0.42603550295857995, 'anger': 0.3401360544217687, 'disgust': 0.4868035190615836, 'sadness': 0.5694444444444444, 'shame': 0.4276315789473685}
----------------------------------------------------

joy
anger
fear
sadness
guilt
disgust
shame
0.7817750543585689
